 Імпорти та конфіг

In [ ]:
import os, time, torch
from PIL import Image
from diffusers import StableDiffusionPipeline, StableDiffusionLatentUpscalePipeline
from diffusers import DPMSolverMultistepScheduler

Основні налаштування

In [ ]:
#Конфіг
MODEL_ID = "stabilityai/stable-diffusion-2-1"
UPSCALE_ID = "stabilityai/sd-x2-latent-upscaler"  # x2 реалістичний апскейл

OUT_DIR = "outputs"
os.makedirs(OUT_DIR, exist_ok=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float32

Промпти та негативний промпт

In [ ]:
#Промпти
prompts = [
    # 1) Простий
    "Photorealistic night city street in the rain cartoon",

    # 2) Детальніший
    "Photorealistic night city street in the rain, wet asphalt reflecting neon lights, people with umbrellas walking along the sidewalk, realistic lighting and shadows",

    # 3) Найдетальніший
    "Photorealistic night city street in the rain, wet asphalt reflecting colorful neon signs and streetlights, people with umbrellas walking along the busy sidewalk, shop windows glowing warmly, realistic lighting and shadows, cinematic composition, ultra realistic, high resolution, natural color grading, detailed reflections, physically based lighting, depth of field, 35mm photo, HDR"
]

negative_prompt = (
    "low quality, blurry, distorted, artifacts, watermark, text, logo, oversaturated, overexposed, "
    "jpeg artifacts, grainy, worst quality, lowres, deformed, bad anatomy, duplicate objects, tiling"
)

Параметри генерації

In [ ]:
#Параметри генерації 
guidance_scale = 7.8
num_inference_steps = 48
height, width = 1024, 768  # безпечніше за 1024x768 на 8ГБ, але значно чіткіше, ніж 768x512

seed = 42
generator = torch.Generator(device=device).manual_seed(seed)

Завантаження базового пайплайну SD 2.1

In [ ]:
#Базовий пайплайн
pipe = StableDiffusionPipeline.from_pretrained(
    MODEL_ID,
    torch_dtype=dtype,
    use_safetensors=True
).to(device)

# Scheduler на різкіший 
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

# Оптимізації пам’яті 
pipe.enable_attention_slicing()      # менше VRAM на self-attention
pipe.enable_vae_slicing()            # менше VRAM у VAE
pipe.enable_vae_tiling()             # тайлінг в VAE для великих розмірів
try:
    pipe.enable_xformers_memory_efficient_attention()
except Exception:
    pass  

Генерація базових зображень

In [ ]:
base_results = []
for i, prompt in enumerate(prompts, start=1):
    t0 = time.time()
    image = pipe(
        prompt=prompt,
        negative_prompt=negative_prompt,
        guidance_scale=guidance_scale,
        num_inference_steps=num_inference_steps,
        height=height,
        width=width,
        generator=generator
    ).images[0]

    fname = f"beach{i:02d}.png"
    path = os.path.join(OUT_DIR, fname)
    image.save(path)
    dt = time.time() - t0
    base_results.append((i, path, round(dt, 1)))
    print(f"[BASE {i}] Saved: {path} | {dt:.1f}s")


Апскейл 

In [ ]:
#x2 Latent Upscale для більш реалістичної мікротекстури
up_pipe = StableDiffusionLatentUpscalePipeline.from_pretrained(
    UPSCALE_ID,
    torch_dtype=dtype
).to(device)

up_pipe.enable_attention_slicing()
up_pipe.enable_vae_slicing()
up_pipe.enable_vae_tiling()

up_results = []
for i, path, _ in base_results:
    img = Image.open(path).convert("RGB")
    t0 = time.time()
    up_img = up_pipe(
        prompt=prompts[i-1],                # промпт
        image=img,                          # вхід — базове зображення
        num_inference_steps=20,             
        guidance_scale=0.0                  # апскейлер
    ).images[0]

    up_path = os.path.join(OUT_DIR, f"img_up_{i:02d}.png")
    up_img.save(up_path)
    dt = time.time() - t0
    up_results.append((i, up_path, round(dt, 1)))
    print(f"[UP x2 {i}] Saved: {up_path} | {dt:.1f}s")
